In [1]:
import robustbench
from robustbench.utils import load_model
from tqdm import tqdm
import os
import torch
from autoattack import AutoAttack
from robustbench.data import _load_dataset
from torchvision import transforms, datasets
import robustbench
from robustbench.utils import load_model
import os
import torch
import multiprocessing as mp

In [5]:
pip install easydict

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6360 sha256=adf56cfe0a7250aa13f10a3e01631c4b737ea56215c6ed408daa09283b4aa820
  Stored in directory: /tmp/pip-ephem-wheel-cache-7o6tdnu9/wheels/d3/e0/e9/305e348717e399665119bd012510d51ff4f22d709ff60c3096
Successfully built easydict
Note: you may need to restart the kernel to use updated packages.


In [2]:
def replace_module(module, filter_fn, replacement_fn):
    for name, child in module.named_children():
        if filter_fn(child):
            replacement = replacement_fn(child)
            setattr(module, name, replacement)
            
    for child in module.children():
        replace_module(child, filter_fn, replacement_fn)

onnx_opset = 13        
        
for dataset in list(robustbench.model_zoo.enums.BenchmarkDataset):
    
    if dataset.value == "imagenet":
        x = torch.randn(256, 3, 224, 224)
    else:
        x = torch.randn(256, 3, 32, 32)
        
    for paper_id, infos in tqdm(robustbench.model_zoo.models.model_dicts[dataset][robustbench.model_zoo.enums.ThreatModel.Linf].items()):
        path = f"/ssd1/robustness_onnx_zoo/robust/robustbench_{paper_id}_{dataset.value}_linf_{onnx_opset}.onnx"
        
        if not os.path.exists(path):
            try:
                model = load_model(model_name=paper_id, dataset=dataset.value, threat_model="Linf")
                replace_module(model, lambda m: type(m) == robustbench.model_zoo.architectures.dm_wide_resnet.Swish, lambda m: torch.nn.SiLU())
                with torch.no_grad():
                    torch.onnx.export(model, x, path, export_params=True, opset_version=onnx_opset)
            except Exception as e:
                msg = str(e)
                if "\n" in msg:
                    msg = msg[:msg.index("\n")]
                print(dataset.value, paper_id, type(e), msg)

  0% 0/51 [00:00<?, ?it/s]Warning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied.
100% 51/51 [00:23<00:00,  2.18it/s]
100% 17/17 [00:00<00:00, 27319.22it/s]

cifar10 Kang2021Stable <class 'RuntimeError'> Exporting the operator tril to ONNX opset version 13 is not supported. Support for this operator was added in version 14, try exporting with this version.



100% 6/6 [00:00<00:00, 13654.81it/s]
